# Hilbert Detector

In [1]:
# first let's load in all our packages
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import re
import pandas as pd


from mne_bids import (read_raw_bids, BIDSPath, 
                      get_entity_vals, get_datatypes, 
                      make_report)
from mne_bids.stats import count_events

import mne
from mne import make_ad_hoc_cov

basepath = os.path.join(os.getcwd(), "../..")
sys.path.append(basepath)
from mne_hfo import HilbertDetector

In [2]:
%%capture
# this may change depending on where you store the data
root = "C:/Users/patri/Dropbox/fedele_hfo_data"
subjects = get_entity_vals(root, 'subject')
sessions = get_entity_vals(root, 'session')
subjectID = subjects[0]
sessionID = sessions[0]
bids_path = BIDSPath(subject=subjectID, session=sessionID,
                     datatype='ieeg', 
                     suffix='ieeg',
                     extension='.vhdr', root=root)

# get first matching dataset
fpath = bids_path.match()[0]
# load dataset into mne Raw object
extra_params = dict(preload=True)
raw = read_raw_bids(fpath, extra_params)

In [3]:
def convert_to_bipolar(raw, drop_originals=True):
    original_ch_names = raw.ch_names
    ch_names_sorted = sorted(original_ch_names)
    ch_pairs = [] 
    for first, second in zip(ch_names_sorted, ch_names_sorted[1:]):
        firstName = re.sub(r'[0-9]+', '', first)
        secondName = re.sub(r'[0-9]+', '', second)
        if firstName == secondName:
            ch_pairs.append((first,second))
    for ch_pair in ch_pairs:
        raw = mne.set_bipolar_reference(raw, ch_pair[0], ch_pair[1], drop_refs=False)
    if drop_originals:
        raw = raw.drop_channels(original_ch_names)
    return raw

In [4]:
%%capture
raw = convert_to_bipolar(raw)


TypeError: slice indices must be integers or None or have an __index__ method

In [7]:
info = raw.info
sfreq = info['sfreq']
data = raw.get_data(start=0, stop=int(30*sfreq))
raw = mne.io.RawArray(data, info=info)

Creating RawArray with float64 data, n_channels=43, n_times=60000
    Range : 0 ... 59999 =      0.000 ...    30.000 secs
Ready.


In [8]:
kwargs = {
    'band_method': 'log',
    'filter_band': (80, 250), # (l_freq, h_freq)
    'threshold': 3, # Number of st. deviations
    'hfo_name': "ripple",
    'n_jobs': 1,
}
hil_detector = HilbertDetector(**kwargs)

In [9]:
raw.n_times

60000

In [10]:
# perform the fits
hil_detector = hil_detector.fit(raw)

  0%|          | 0/43 [00:00<?, ?it/s]

freq_span: 61


  2%|▏         | 1/43 [00:00<00:21,  2.00it/s]

freq_span: 61


  5%|▍         | 2/43 [00:00<00:20,  2.00it/s]

...

 95%|█████████▌| 41/43 [00:18<00:00,  2.26it/s]

freq_span: 61


 98%|█████████▊| 42/43 [00:19<00:00,  2.26it/s]

freq_span: 61


100%|██████████| 61/61 [00:00<00:00, 815.07HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 848.13HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 830.25HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 847.72HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 847.64HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 871.42HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 865.06HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 803.96HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 848.21HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 884.18HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 884.58HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 909.52HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 896.36HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 896.99HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 883.76HFO-first-phase/s]

100%|██████████| 61/61 [00:00<00:00, 872.18HFO-first-ph

In [12]:
hil_detector.df_

,onset,duration,label,channels,sample
0,10.4095,0.6320,ripple,AHR1-AHR2,20819.0
1,14.2345,0.2995,ripple,AHR1-AHR2,28469.0
2,16.0610,0.3110,ripple,AHR1-AHR2,32122.0
3,23.8995,0.4905,ripple,AHR1-AHR2,47799.0
4,10.4225,0.5130,ripple,AHR2-AHR3,20845.0
...,...,...,...,...,...
69,16.0450,0.5025,ripple,PHR4-PHR5,32090.0
70,23.9560,0.5325,ripple,PHR4-PHR5,47912.0
71,6.4010,0.1840,ripple,PHR5-PHR6,12802.0
72,1.4250,0.1630,ripple,PHR7-PHR8,2850.0
